In [ ]:
!pip install peft py7zr >> /dev/null

In [1]:
from datasets import (
    load_dataset,
    concatenate_datasets
)
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
)

dataset = load_dataset('samsum')

model_id = "google/flan-t5-xxl"

tokenizer = AutoTokenizer.from_pretrained(model_id)

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/3f7dba43be72ab10ca66a2e0f8547b3590e96c2bd9f2cbb1f6bb1ec1f1488ba6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [10]:
# Abstractive Summarization is a text-generation task
import numpy as np
concated_ds = concatenate_datasets([dataset['train'],
                                        dataset['test']])

/opt/conda/lib/python3.10/site-packages/datasets/table.py:1317: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [15]:
def tokenized_dialog(row):
    return tokenizer(row["dialogue"],
              truncation=True)
    
def tokenized_summary(row):
    return tokenizer(row["summary"],
              truncation=True)

In [16]:
tokenized_inputs = concated_ds.map(tokenized_dialog,
                                   batched=True,
                                   num_proc=4,
                                   remove_columns=['id','dialogue','summary'])

tokenized_outputs = concated_ds.map(tokenized_summary,
                                    batched=True,
                                    num_proc=4,
                                    remove_columns=['id','dialogue','summary'])

/opt/conda/lib/python3.10/site-packages/datasets/table.py:1283: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


/opt/conda/lib/python3.10/site-packages/datasets/table.py:1283: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


/opt/conda/lib/python3.10/site-packages/datasets/table.py:1283: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


/opt/conda/lib/python3.10/site-packages/datasets/table.py:1283: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


#1:   0%|          | 0/4 [00:00<?, ?ba/s]

...

/opt/conda/lib/python3.10/site-packages/datasets/table.py:1283: FutureWarning: promote has been superseded by promote_options='default'.
  return cls._concat_blocks(pa_tables_to_concat_vertically, axis=0)


#0:   0%|          | 0/4 [00:00<?, ?ba/s]

#2:   0%|          | 0/4 [00:00<?, ?ba/s]

#1:   0%|          | 0/4 [00:00<?, ?ba/s]

#3:   0%|          | 0/4 [00:00<?, ?ba/s]

In [17]:
tokenized_inputs

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 15551
})

In [18]:
inp_length = [len(x) for x in tokenized_inputs['input_ids']]
max_source_length = int(np.percentile(inp_length, 85))
output_length = [len(x) for x in tokenized_outputs['input_ids']]
max_tgt_length = int(np.percentile(output_length, 90))

a = np.array([[10, 7, 4], [3, 2, 1]])

np.percentile(a, 50)

3.5

np.percentile(a, 50, axis=0)

array([6.5, 4.5, 2.5])

np.percentile(a, 50, axis=1)

array([7.,  2.])

np.percentile(a, 50, axis=1, keepdims=True)

array([[7.],

In [19]:
print(max_source_length)
print(max_tgt_length)

255
50


In [20]:
tokenizer.pad_token

'<pad>'

In [21]:
tokenizer.pad_token_id

0

In [25]:
def preprocess_dataset(sample, padding='max_length'):
    # assuming, the data is going to be batched
    inputs = [f"summarise {item}" for item in sample["dialogue"]]
    
    model_ins = tokenizer(inputs,
                          max_length=max_source_length,
                         padding=padding,
                         truncation=True)
    labels = tokenizer(sample['summary'],
                       max_length=max_tgt_length,
                      padding=padding,
                      truncation=True)
    
    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ] # if it is not pad_token_id then leave it as it is, else replace with -100\
    
    model_ins['labels'] = labels["input_ids"]
    
    return model_ins

In [26]:
lm_das = dataset.map(preprocess_dataset,
                    batched=True,
                    num_proc=6,
                    remove_columns=["id", "dialogue",
                                   "summary"])
lm_das

#1:   0%|          | 0/3 [00:00<?, ?ba/s]

#0:   0%|          | 0/3 [00:00<?, ?ba/s]

#3:   0%|          | 0/3 [00:00<?, ?ba/s]

#4:   0%|          | 0/3 [00:00<?, ?ba/s]

#2:   0%|          | 0/3 [00:00<?, ?ba/s]

...

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#4:   0%|          | 0/1 [00:00<?, ?ba/s]

#5:   0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [27]:
from transformers import AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_id,
                                             load_in_8bit=True,
                                             device_map="auto")

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
from peft import (
    LoraConfig,
    get_peft_model,
    TaskType,
    prepare_model_for_kbit_training,
    prepare_model_for_int8_training
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

int_model = prepare_model_for_int8_training(model)

qt_model = get_peft_model(int_model, lora_config) 

qt_model.print_trainable_parameters()

In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="lora-flan-t5-xxl"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
	auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

In [ ]:
trainer.train()
# trainer.model.base_model.save_pretrained(peft_model_id)

In [ ]:
local_path = "~/training_models/lora_t5_samsum_training"
peft_model_id="results"
trainer.model.save_pretrained(local_path)
tokenizer.save_pretrained(local_path)

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "results"
config = PeftConfig.from_pretrained(local_path)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,
                                              load_in_8bit=True,
                                              device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model,
                                  local_path,
                                  device_map={"":0})
model.eval()

print("Peft model loaded")

In [ ]:
from datasets import load_dataset
from random import randrange


# Load dataset from the hub and get a sample
dataset = load_dataset("samsum")
sample = dataset['test'][randrange(len(dataset["test"]))]

input_ids = tokenizer(sample["dialogue"],
                      return_tensors="pt",
                      truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids,
                         max_new_tokens=10,
                         do_sample=True,
                         top_p=0.9)

print(f"input sentence: {sample['dialogue']}\n{'---'* 20}")

print(f"summary:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

Kommonly used metrics to evaluate summarization task is rogue_score short for Recall-Oriented Understudy for Gisting Evaluation

In [ ]:
import evaluate
import numpy as np
from datasets import load_from_disk
from tqdm import tqdm

# Metric
metric = evaluate.load("rouge")

def evaluate_peft_model(sample,max_target_length=50):
    # generate summary
    outputs = model.generate(input_ids=sample["input_ids"].unsqueeze(0).cuda(),
                             do_sample=True,
                             top_p=0.9,
                             max_new_tokens=max_target_length)
    prediction = tokenizer.decode(outputs[0].detach().cpu().numpy(),
                                  skip_special_tokens=True)
    # decode eval sample
    # Replace -100 in the labels as we can't decode them.
    
    labels = np.where(sample['labels'] != -100,
                      sample['labels'],
                      tokenizer.pad_token_id)
    
    labels = tokenizer.decode(labels,
                              skip_special_tokens=True)

    # Some simple post-processing
    return prediction, labels

# load test dataset from distk
test_dataset = load_from_disk("data/eval/").with_format("torch")

# run predictions
# this can take ~45 minutes
predictions, references = [] , []
for sample in tqdm(test_dataset):
    p,l = evaluate_peft_model(sample)
    predictions.append(p)
    references.append(l)

# compute metric
rogue = metric.compute(predictions=predictions, references=references, use_stemmer=True)

# print results
print(f"Rogue1: {rogue['rouge1']* 100:2f}%")
print(f"rouge2: {rogue['rouge2']* 100:2f}%")
print(f"rougeL: {rogue['rougeL']* 100:2f}%")
print(f"rougeLsum: {rogue['rougeLsum']* 100:2f}%")